<a href="https://colab.research.google.com/github/e9anna/book_algorithm/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EA%B5%AC%ED%98%84/%EC%8B%A4%ED%97%98/TF_IDF_%EC%8B%A4%ED%97%980913_%EB%8D%B0%EB%AA%A8%EB%8D%B0%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9/13일자 초안 코드

- 수정자: 조은정

# 필요한 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# 데이터

In [ ]:
docu = pd.read_csv('/content/tf실험.csv', encoding='utf-8')
docu.head()

df = open('/content/개미.txt', 'r', encoding='utf-8')
text = df.read()

# 데이터 클렌징

1) konlpy 와 사이킷런( sklearn)의 TfidfVectorizer 라이브러리로 
- 책 소개에 있는 글자 외의 다른 기호 등 제거
- 명사만 추출하기


2) 추출한 명사를 열4 (명사)에 추가

In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 48.4 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 448 kB 53.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
# 1) 필요한 라이브러리
from konlpy.tag import Okt
from collections import Counter 

In [7]:
def get_keywords(origin):
  origin_text = origin['책소개'].values.tolist() 
  noun_list = [] # 비어있는 리스트 하나 소환
  for i in range(len(origin_text)):
    okt = Okt()
    noun1 = okt.nouns(origin_text[i]) # 1) 책 소개에서 명사만 추출하게 okt 라이브러리 사용
    noun_list.append(noun1) # 비어있는 리스트에 명사추출한거 붙여넣기
  df = pd.DataFrame(origin, columns=['제목', '작가', '책소개']) # df라는 새로운 데이터프레임 만들기
  df.insert(3,'명사추출',noun_list, True)  # 2) 추출한 명사를 4번째 열에 추가
  return df

get_keywords(docu) # 파라미터 자리에 위에서 불러온 csv파일 이름 적기

,제목,작가,책소개,명사추출
0,개미,베르나르 베르베르,베르나르 베르베르의 장편소설 [개미] 시리즈. 저자는 기상천외한 개미의 세계를 개미...,"[베르나르, 베르베르, 장편소설, 개미, 시리즈, 저자, 기상, 외한, 개미, 세계..."
1,관계의 힘,레이먼즈 조,진정한 성공과 행복으로 이끄는 인간관계의 힘 “대한민국 직장인들이 직장생활에서 가장...,"[성공, 행복, 인간관계, 힘, 대한민국, 직장인, 직장, 생활, 가장, 것, 일,..."
2,이기적 유전자,리처드 도킨스,"세계적 베스트셀러, 과학을 넘어선 우리 시대의 고전, 『이기적 유전자』의 40주년 ...","[세계, 베스트셀러, 과학, 우리, 시대, 고전, 이기, 유전자, 의, 주년, 기념..."


In [8]:
docu2 = get_keywords(docu)
docu2.head()

,제목,작가,책소개,명사추출
0,개미,베르나르 베르베르,베르나르 베르베르의 장편소설 [개미] 시리즈. 저자는 기상천외한 개미의 세계를 개미...,"[베르나르, 베르베르, 장편소설, 개미, 시리즈, 저자, 기상, 외한, 개미, 세계..."
1,관계의 힘,레이먼즈 조,진정한 성공과 행복으로 이끄는 인간관계의 힘 “대한민국 직장인들이 직장생활에서 가장...,"[성공, 행복, 인간관계, 힘, 대한민국, 직장인, 직장, 생활, 가장, 것, 일,..."
2,이기적 유전자,리처드 도킨스,"세계적 베스트셀러, 과학을 넘어선 우리 시대의 고전, 『이기적 유전자』의 40주년 ...","[세계, 베스트셀러, 과학, 우리, 시대, 고전, 이기, 유전자, 의, 주년, 기념..."


### 명사 빈도 셀 때 쓰는코드
- 필요없지만 이후에 혹시나 참고용으로..

#### **명사 빈도 카운트 할 때 쓰는 코드**
list_docu = docu['책소개'].values.tolist()

#### **okt 객체 생성**
okt = Okt()
noun = okt.nouns(list_docu[0])
count = Counter(noun)

#### **제일 자주쓰는 명사 10개까지만**
noun_list = count.most_common(10)
for i in noun_list:
  print(i)

# 유사도 측정

1) 검색하고 싶은 키워드를 넣었을 때 가장 비슷한 키워드는 무엇이 나올까?

2) 그렇게 해서 나온 키워드가 가장 많이 포함되어 있는 책 추천하기

## 1. 검색하고 싶은 키워드를 입력해서 나오는 가장 비슷한 키워드는? 

### 1) 단어 매트릭스 만들기

In [9]:
documents = docu2['명사추출'].values[0]
documents # list 형식

['베르나르',
 '베르베르',
 '장편소설',
 '개미',
 '시리즈',
 '저자',
 '기상',
 '외한',
 '개미',
 '세계',
 '개미',
 '시각',
 '흥미',
 '사랑',
 '반역',
 '생존',
 '위',
 '투쟁',
 '이',
 '책',
 '우선',
 '표면',
 '소설',
 '설정',
 '인간',
 '사회',
 '개미',
 '사회',
 '실종',
 '살인',
 '등',
 '추적',
 '그',
 '원인',
 '추리',
 '소설',
 '양식',
 '개미',
 '생태',
 '등',
 '과학',
 '위',
 '관찰',
 '묘사',
 '과학',
 '소설',
 '양식',
 '번',
 '데',
 '감각',
 '재미']

In [10]:
# sklearn에 있는 TfidfVectorizer 라이브러리 사용
tfidf = TfidfVectorizer(max_features = 100, max_df=0.95, min_df=0)

# tf-idf의 term-document matrix 생성하기
tfidf_mt = tfidf.fit_transform(documents) 

# tf-idf dictionary 만들기
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)

['감각', '개미', '과학', '관찰', '기상', '묘사', '반역', '베르나르', '베르베르', '사랑', '사회', '살인', '생존', '생태', '설정', '세계', '소설', '시각', '시리즈', '실종', '양식', '외한', '우선', '원인', '인간', '장편소설', '재미', '저자', '추리', '추적', '투쟁', '표면', '흥미']


In [11]:
data_array = tfidf_mt.toarray()
data = pd.DataFrame(data_array, columns=tfidf_dict)
data.shape

(51, 33)

- 'tf실험' csv 파일의 1행의 명사추출 열을 TfidfVectorizer 라이브러리를 활용하여 매트릭스화 시키기: (51x33) 크기의 매트릭스

In [12]:
count = Counter(documents)
# 단어빈도(count.get)를 기준으로 내림차순 정렬
vocab = sorted(count, key=count.get, reverse=True)
vocab

['개미',
 '소설',
 '위',
 '사회',
 '등',
 '양식',
 '과학',
 '베르나르',
 '베르베르',
 '장편소설',
 '시리즈',
 '저자',
 '기상',
 '외한',
 '세계',
 '시각',
 '흥미',
 '사랑',
 '반역',
 '생존',
 '투쟁',
 '이',
 '책',
 '우선',
 '표면',
 '설정',
 '인간',
 '실종',
 '살인',
 '추적',
 '그',
 '원인',
 '추리',
 '생태',
 '관찰',
 '묘사',
 '번',
 '데',
 '감각',
 '재미']

In [13]:
#단어들에 번호를 매겨 그 번호와 그 단어들을 dict 형태로 저장
word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}
word2idx

# 많이 나오는 단어 순대로 중요함

{'감각': 39,
 '개미': 1,
 '과학': 7,
 '관찰': 35,
 '그': 31,
 '기상': 13,
 '데': 38,
 '등': 5,
 '묘사': 36,
 '반역': 19,
 '번': 37,
 '베르나르': 8,
 '베르베르': 9,
 '사랑': 18,
 '사회': 4,
 '살인': 29,
 '생존': 20,
 '생태': 34,
 '설정': 26,
 '세계': 15,
 '소설': 2,
 '시각': 16,
 '시리즈': 11,
 '실종': 28,
 '양식': 6,
 '외한': 14,
 '우선': 24,
 '원인': 32,
 '위': 3,
 '이': 22,
 '인간': 27,
 '장편소설': 10,
 '재미': 40,
 '저자': 12,
 '책': 23,
 '추리': 33,
 '추적': 30,
 '투쟁': 21,
 '표면': 25,
 '흥미': 17}

In [14]:
tsne = TSNE(n_components=2, n_iter=10000, verbose=1)

In [15]:
data_array.shape

(51, 33)

In [16]:
data_array.T.shape

(33, 51)

In [17]:
# Z : 위의 51X33차원에서 33차원에 존재하는 51개의 단어들을 2차원에 51개의 단어로 표현하기
Z = tsne.fit_transform(data_array.T)

[t-SNE] Computing 32 nearest neighbors...
[t-SNE] Indexed 33 samples in 0.000s...
[t-SNE] Computed neighbors for 33 samples in 0.012s...
[t-SNE] Computed conditional probabilities for sample 33 / 33
[t-SNE] Mean sigma: 1.046431
[t-SNE] KL divergence after 250 iterations with early exaggeration: 44.130917
[t-SNE] KL divergence after 3250 iterations: 0.496667


### 2) 단어들의 관계를 벡터 연산하기

- w1 -> 임의로 '과학' 이라는 키워드를 아래 샘플에서 입력
- w2 -> 임의로 '소설' 이라는 키워드를 아래 샘플에서 입력
- w3 -> 임의로 '시리즈' 라는 키워드를 아래 샘플에서 입력

- emb : 임베딩 시키는 건 위의 Z에서 이미 트랜스폼 시킴

- word2idx 파라미터 자리에 -> best_word: 가장 비슷한 단어는?

In [18]:
def find_analogies(w1, w2, w3, emb, word2idx):
    pos_word1 = emb[word2idx[w1]]
    pos_word2 = emb[word2idx[w2]]
    neg_word1 = emb[word2idx[w3]]
    v0 = pos_word1 + pos_word2 - neg_word1

    def dist1(a, b):
        return np.linalg.norm(a - b)
    def dist2(a, b):
        return 1 - a.dot(b) / (np.linalg.norm(a) * np.linalg.norm(b))

    for dist, name in [(dist1, 'Euclidean'), (dist2, 'cosine')]:
        min_dist = float('inf')
        best_word = ''
        for word, idx in iteritems(word2idx):
            if word not in (w1, w2, w3):
                v1 = emb[idx]
                d = dist(v0, v1)
                if d < min_dist:
                    min_dist = d
                    best_word = word
        print("closest match by", name, "distance:", best_word)
        print(w1, "-", w2, "=", w3, "-", best_word)

In [19]:
analogies_to_try = (
    ('과학', '소설','시리즈'),
)

word2idx = {word: ii for ii, word in enumerate(tfidf.get_feature_names())}
word2idx

for word_list in analogies_to_try:
    print(word_list)
    w1, w2, w3 = word_list
    find_analogies(w1=w1, w2=w2, w3=w3, emb=Z, word2idx=word2idx)

('과학', '소설', '시리즈')
closest match by Euclidean distance: 표면
과학 - 소설 = 시리즈 - 표면
closest match by cosine distance: 표면
과학 - 소설 = 시리즈 - 표면


- 3개의 키워드 ('과학', '소설','시리즈' ) 를 입력했을때 유클리디안 거리로 가장 가까운 단어는 원인
- 코사인 거리로 가장 가까운 단어는 개미